# B-Tree

B-tree is a common data structure for managing data stored in hard disks. Because of the mechanincal movements of hard disks (arm moving and disk spining), it is most economical to bring a large chunk of data stored consecutively into the main memory in one disk-read operation and write a large chunkof data to a disk in one disk-read operation. This chunk of data is called a page. The size of one page ranges from $2^{11} = 2,048$ bytes to $2^{14} = 16,384$ bytes. B-tree stores one page of data in one node. To facilitate searching, each internal node can hold multiple keys. To cut down disk accesses, we want a B-tree to be short.

<img src="Disk.png" width="300"/>

The typical pattern for working with an object is as follows: 

$x$ is a pointer to some object
DISK-READ($x$)
operations that access and/or modify the attributes of $x$
DISK-WRITE($x$) (omitted if no attributes of x were changed)
other operations that access but do not modify attributes of $x$

In a B-tree, as in any form of BST, any "satellite information” (data) associated with a key resides in the same node as the key. Or we may think of a key as a point to the data.

In a variant of B-tree called B$^+$-tree, all the satellite information (data) is stored in the leaves and the internal nodes only store keys and child pointers. This maximizes the branching factor of the internal nodes.

## B-tree definition 

A tree $T$ is a B-tree if it satisfies the following conditions:

1. All leaves are at the same level.
2. Let $t\geq 2$ be a positive integer called a minimum degree, whose value is determined by the page size.
3. Every node except the root must contain at least $t-1$ keys. If $T$ is not empty, then the root contains one or more keys.
4. All nodes (the root included) may contain upto $2t – 1$ keys. A node is full if it contains exactly $2t-1$ keys.
5. Each internal node of $k$ keys has $k+1$ children. Thus, an internal node may have at most $2t$ children. 
6. Keys of a node are sorted in increasing order. The child between two keys $k_1$ and $k_2$ contains all keys in the range between $k_1$ and $k_2$.


The following is an example of 2-3-4 B-tree, i.e., each node has 2, 3, or 4 children:

<img src="Fig18-1.png" width="600"/>



## Properties

1. B-Tree grows and shrinks from the root. This is different from BST, which grows downward and shrinks upward.
2. Inserting a new node in B-tree happens only at the leave level.
3. Like other balanced BST such as AVL and red-black trees, each operation of search, insert, and delete incurs $O(\log_t n) = O(\log n)$ disk accesses (see below), since the minimum degree $t$ is a constant.

Let $h$ be the height of a B-tree $T$ with a minimum degree $t$. Let $n$ be the total number of keys in $T$. Then 
$$h \leq \log_t \frac{n+1}{2}.$$

Proof. 
\begin{align*}
n \geq 1 + (2t + 2t^2 + \cdots + 2t^{h-1})(t-1) = 1 + \frac{2(t^h - 1)}{t-1}(t-1) = 2t^h -1.
\end{align*}

## Applications of B-Trees

1. Searching for data in a dataset stored on a hard disk can be achieved in significantly less time using the B-Tree
2. Most servers use the B-tree to manage data.
3. B-Trees are used in natural language processing, computer networks, and cryptography, among others.

## B-tree search and insertion

1. Without loss of generality, assume that the root of the B-tree is always in main memory, so that no disk-read on the root is needed. We only need to perform a disk-write of the root whenever the root node is changed.

2. Any nodes that are passed as parameters must already have had a disk-read operation performed on them.

3. Inserting a new key to a full root causes the root to split to a new root, and hence the height of the tree is incremented by 1. This is different from BST insertion.

<img src="Split.png" width="600"/>

<img src="Insertion.png" width="600"/>

In [1]:
# Create a node
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.child = []
        
# BTree
class BTree:
    def __init__(self, t): # t is the minimum degree
        self.root = BTreeNode(True)
        self.t = t

    # Search key k starting from x
    # If x is not specified, search k starting from the root
    def search_key(self, k, x=None):
        if x is None:
            return self.search_key(k, self.root)
        else: # x is a current node
            i = 0
            while i < len(x.keys) and k > x.keys[i][0]:
                i += 1
            if i < len(x.keys) and k == x.keys[i][0]:
                return (x, i) # found k
            elif x.leaf: # if k is not in x and x has no children
                return None
            else:
                return self.search_key(k, x.child[i])
               
    # Insert key k, which is in th form of a pair key and content
    def insert(self, k):
        root = self.root
        if len(root.keys) == 2 * self.t - 1: # full--no keys are available
            temp = BTreeNode()
            self.root = temp
            temp.child.insert(0, root) 
            #list insert: at location 0, insert root; that is, temp points to root
            self.split_child(temp, 0)
            self.insert_non_full(temp, k)
        else:
            self.insert_non_full(root, k)
            
    # Split the child x at location i
    def split_child(self, x, i): 
        t = self.t # minimum degree
        y = x.child[i]
        z = BTreeNode(y.leaf)
        x.child.insert(i + 1, z) # list insert
        x.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t: (2 * t) - 1]
        y.keys = y.keys[0: t - 1]
        if not y.leaf:
            z.child = y.child[t: 2 * t]
            y.child = y.child[0: t - 1]

    # Insert nonfull
    def insert_non_full(self, x, k):
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append((None, None)) # add a new key pair holder (None, None)
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]
                i -= 1
            x.keys[i + 1] = k # insert k
        else:
            while i >= 0 and k[0] < x.keys[i][0]:
                i -= 1
            i += 1
            if len(x.child[i].keys) == (2 * self.t) - 1:
                self.split_child(x, i)
                if k[0] > x.keys[i][0]:
                    i += 1
            self.insert_non_full(x.child[i], k)

    # Print the tree
    def print_tree(self, x, l=0):
        print("Level ", l, " ", len(x.keys), end=":")
        for i in x.keys:
            print(i, end=" ")
        print()
        l += 1
        if len(x.child) > 0:
            for i in x.child:
                self.print_tree(i, l)

In [10]:
# driver code

B = BTree(3) # t = 3
    
for i in range(10):
    B.insert((i, 10 + 2 * i))

B.print_tree(B.root)

if B.search_key(8) is not None:
    print("\nFound")
else:
    print("\nNot Found")

if B.search_key(20) is not None:
    print("\nFound")
else:
    print("\nNot Found")

Level  0   2:(2, 14) (5, 20) 
Level  1   2:(0, 10) (1, 12) 
Level  1   2:(3, 16) (4, 18) 
Level  1   4:(6, 22) (7, 24) (8, 26) (9, 28) 

Found

Not Found


## Delete a key from a B-tree

Deleting a key from a B-tree consists of three main events: 
1. Determine by searching if a node that contains the key to be deleted exists.
2. Delete the key from the node that contains it.
3. Balance the tree if needed.

B-tree deletion is more complex than red-black tree deletion.

Underflow occurs when a node contains less than the minimum number of keys it should hold. 

Two terminologies:
1. Inorder Predecessor of a node is the largest key on the left child of the node.
2. Inorder Successor of a node is the smallest key on the right child of the node.

### Case I: The key to be deleted is in the leaf. There are two sub-cases:

<b>I-1</b>: The node containing the key to be deleted has more than $t-1$ keys. Thus, deleting the key is straightforward, for it does not violate the property of the minimum number of keys a node should hold. In the tree below, deleting 32 does not violate the above properties, where $t = 2$.

<img src="delete-leaf-1.webp" width="450"/>

<b>I-2</b>: The node containing the key to be deleted has $t - 1$ keys. Thus, deleting the key violates the property of the minimum number of keys a node should hold. 

To maintain the property of B-tree, we first check if its immediate left sibling has at least $t$ keys, if so, borrow a key from it. If not, check if its immediate right sibling
has at least $t$ keys, if so, borrow a key from it. 

In the tree below, deleting 31 yields a violation, and a key from the left sibling can be borrowed.

<img src="delete-leaf-2.webp" width="450"/>

If both the immediate sibling nodes have $t-1$ keys, then merge the node with either the left sibling node or the right sibling node. This merging is done through the parent node.

<img src="delete-leaf-3.webp" width="450"/>

### Case II: The key to be deleted is in an internal node

<b>II-1</b>: The internal node containing the key to be deleted is replaced by an inorder predecessor if the key's left child has more than $t-1$ keys.

<img src="delete-internal-1.webp" width="450"/>


<b>II-2</b> The internal node containing the key to be deleted, is replaced by an inorder successor if the right child of the key has more than $t-1$ keys. This similar to case II.1.

<b>II-3</b> Both children of the key have exactly $t-1$ keys, then merge the left and the right children.

<img src="delete-internal-2.webp" width="450"/>

After merging, if the parent node has less than $t-1$ keys, then look for its siblings as in Case I.

### Case III:  The height of the tree is reduced 

If the target key is in an internal node, and the deletion of the key leads to less than $t-1$ nodes, then look for the key's left child (inorder predecessor) and the key's right child (inorder successor). If both children contain $t-1$ keys, then merge the children as in Case II-3. 

Again, look for the sibling to borrow a key. But, if the sibling also has only $t-1$ keys, then merge the node with the sibling along with the parent. Arrange the children accordingly (increasing order).

<img src="delete-internal_3.webp" width="450"/>

In [3]:
# Delete a node
def delete(self, x, k):
    t = self.t
    i = 0
    while i < len(x.keys) and k[0] > x.keys[i][0]:
        i += 1
    if x.leaf:
        if i < len(x.keys) and x.keys[i][0] == k[0]:
            x.keys.pop(i)
            return
        return

    if i < len(x.keys) and x.keys[i][0] == k[0]:
        return self.delete_internal_node(x, k, i)
    elif len(x.child[i].keys) >= t:
        self.delete(x.child[i], k)
    else:
        if i != 0 and i + 2 < len(x.child):
            if len(x.child[i - 1].keys) >= t:
                self.delete_sibling(x, i, i - 1)
            elif len(x.child[i + 1].keys) >= t:
                self.delete_sibling(x, i, i + 1)
            else:
                self.delete_merge(x, i, i + 1)
        elif i == 0:
            if len(x.child[i + 1].keys) >= t:
                self.delete_sibling(x, i, i + 1)
            else:
                self.delete_merge(x, i, i + 1)
        elif i + 1 == len(x.child):
            if len(x.child[i - 1].keys) >= t:
                self.delete_sibling(x, i, i - 1)
            else:
                self.delete_merge(x, i, i - 1)
        self.delete(x.child[i], k)

setattr(BTree, 'delete', delete)



In [4]:
# Delete internal node
def delete_internal_node(self, x, k, i):
    t = self.t
    if x.leaf:
        if x.keys[i][0] == k[0]:
            x.keys.pop(i)
            return
        return

    if len(x.child[i].keys) >= t:
        x.keys[i] = self.delete_predecessor(x.child[i])
        return
    elif len(x.child[i + 1].keys) >= t:
        x.keys[i] = self.delete_successor(x.child[i + 1])
        return
    else:
        self.delete_merge(x, i, i + 1)
        self.delete_internal_node(x.child[i], k, self.t - 1)

setattr(BTree, "delete_internal_node", delete_internal_node)          


In [5]:
# Delete the predecessor
def delete_predecessor(self, x):
    if x.leaf:
        return x.pop()
    n = len(x.keys) - 1
    if len(x.child[n].keys) >= self.t:
        self.delete_sibling(x, n + 1, n)
    else:
        self.delete_merge(x, n, n + 1)
    self.delete_predecessor(x.child[n])

setattr(BTree, "delete_predecessor", delete_predecessor)        

In [6]:
# Delete the successor
def delete_successor(self, x):
    if x.leaf:
        return x.keys.pop(0)
    if len(x.child[1].keys) >= self.t:
        self.delete_sibling(x, 0, 1)
    else:
        self.delete_merge(x, 0, 1)
    self.delete_successor(x.child[0])

setattr(BTree, "delete_successor", delete_successor)    

In [7]:
# Delete resolution
def delete_merge(self, x, i, j):
    cnode = x.child[i]

    if j > i:
        rsnode = x.child[j]
        cnode.keys.append(x.keys[i])
        for k in range(len(rsnode.keys)):
            cnode.keys.append(rsnode.keys[k])
            if len(rsnode.child) > 0:
                cnode.child.append(rsnode.child[k])
        if len(rsnode.child) > 0:
            cnode.child.append(rsnode.child.pop())
        new = cnode
        x.keys.pop(i)
        x.child.pop(j)
    else:
        lsnode = x.child[j]
        lsnode.keys.append(x.keys[j])
        for i in range(len(cnode.keys)):
            lsnode.keys.append(cnode.keys[i])
            if len(lsnode.child) > 0:
                lsnode.child.append(cnode.child[i])
        if len(lsnode.child) > 0:
            lsnode.child.append(cnode.child.pop())
        new = lsnode
        x.keys.pop(j)
        x.child.pop(i)

    if x == self.root and len(x.keys) == 0:
        self.root = new

setattr(BTree, "delete_merge", delete_merge)     

In [8]:
# Delete the sibling
def delete_sibling(self, x, i, j):
    cnode = x.child[i]
    if i < j:
        rsnode = x.child[j]
        cnode.keys.append(x.keys[i])
        x.keys[i] = rsnode.keys[0]
        if len(rsnode.child) > 0:
            cnode.child.append(rsnode.child[0])
            rsnode.child.pop(0)
        rsnode.keys.pop(0)
    else:
        lsnode = x.child[j]
        cnode.keys.insert(0, x.keys[i - 1])
        x.keys[i - 1] = lsnode.keys.pop()
        if len(lsnode.child) > 0:
            cnode.child.insert(0, lsnode.child.pop())

setattr(BTree, "delete_sibling", delete_sibling) 

In [12]:
# driver code
B.delete(B.root, (8,))
B.print_tree(B.root)

Level  0   2:(2, 14) (5, 20) 
Level  1   2:(0, 10) (1, 12) 
Level  1   2:(3, 16) (4, 18) 
Level  1   3:(6, 22) (7, 24) (9, 28) 
